# Project Proposal — Heart Disease Prediction

## Introduction

Cardiovascular diseases, particularly heart disease, pose a significant global health challenge. Accurate diagnosis is crucial for effective patient care. In this proposal, we aim to address the question of diagnosing heart disease using machine learning techniques. Specifically, we will attempt to detect the presence of heart disease in patients by leveraging a heart disease dataset obtained from Kaggle. The diagnosis (num) ranges from 0 (no presence) to 4. However, our focus will be on distinguishing between the presence (values 1, 2, 3, 4) and absence (value 0) of heart disease.

The dataset consists of 16 variables, but for the purpose of this project, we will focus on a subset of 6 variables. Below is the description of the selected variables:

1. **Id (id ) : patient Id num
2. **Age (age) **: Age of the patient.
2. **Resting Blood Pressure (trestbps)**: Measured in mm Hg upon hospital admission.
3. **Serum Cholesterol (chol)**: Serum cholesterol level in mg/dL.
4. **Thalach (thalch)**: maximum heart rate achieved
5. **ca** : number of major vessels (0-3) colored by flourosopy
6. **Diagnosis of Heart Disease (presence)**: Angiographic disease status, representing the degree of blockages or narrowing in major vessels.
   - Value P (present): < 50% diameter narrowing
   - Value N (non-present): > 50% diameter narrowing (in any major vessel; attributes 59 through 68 represent vessels)


We aim to answer the following question:

#### Can K-nearest neighbors (KNN) classification accurately diagnose heart disease using only blood pressure, cholesterol levels, blood sugar and number of vessels?


### Preliminary exploratory data analysis

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')
# install.packages("cowplot")
# library(cowplot)
# library(scales)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [ ]:
# install.packages("readr")
# library(readr)

In [39]:

## We are using the heart disease data. As suggested by the professor we downloaded the csv file from kaggle and uploaded it into our notebook

file_path <- "data/heart_disease_uci.csv"

heart_disease_uci <- read_csv(file_path)

# selecting only the rows we need and Change 'num' to a factor
heart_disease_data <- heart_disease_uci |> 
         rename(presence = num) |>
         select(id, age, sex, trestbps, chol, thalch, ca, presence) |>
         mutate(presence = ifelse(presence == 0, "N", "P"),
                presence = as.factor(presence))



# Print the first few rows of the modified dataset
heart_disease_data

set.seed(3456)

# Split the heart disease data into training and testing sets 
heart_disease_split <- initial_split(heart_disease_data, prop = 0.75, strata = presence)  
heart_disease_train <- training(heart_disease_split)   
heart_disease_test <- testing(heart_disease_split)

presence_observations_table <- heart_disease_train |>
    group_by(presence) |>
    summarize(count = n()) 

predictors_mean_table <- heart_disease_train |>
    select(age, trestbps, chol, thalch, ca) |>
    map_df(mean, na.rm = TRUE)

predictors_mean_table
    
missing_number_table <- sum(rowSums(is.na(heart_disease_train)) > 0)

missing_number_table

Rows: 920 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): sex, dataset, cp, restecg, slope, thal
dbl (8): id, age, trestbps, chol, thalch, oldpeak, ca, num
lgl (2): fbs, exang

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,age,sex,trestbps,chol,thalch,ca,presence
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,63,Male,145,233,150,0,N
2,67,Male,160,286,108,3,P
3,67,Male,120,229,129,2,P
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
918,55,Male,122,223,100,NA,P
919,58,Male,NA,385,NA,NA,N
920,62,Male,120,254,93,NA,P


age,trestbps,chol,thalch,ca
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
53.5312,131.9267,197.5571,136.938,0.678733


[1] 469